In [ ]:
# !pip install GML

In [ ]:
# !pip install autofeat

In [ ]:
# !pip install sweetviz

In [ ]:
#!pip install optuna

In [ ]:
import pandas as pd

import numpy as np

from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, PolynomialFeatures, RobustScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.neural_network import MLPRegressor
from sklearn.cluster import KMeans
from sklearn.ensemble import ExtraTreesRegressor, VotingRegressor, HistGradientBoostingRegressor
from sklearn.linear_model import BayesianRidge, LinearRegression, LassoLars, Ridge, ElasticNet, Lasso
from sklearn.inspection import permutation_importance

from mlxtend.regressor import StackingCVRegressor

# import sweetviz as sv

# import smogn

import matplotlib.pyplot as plt

import seaborn as sns

from scipy import stats

from xgboost import XGBRegressor

from catboost import CatBoostRegressor

from lightgbm import LGBMRegressor

import tqdm

from category_encoders import TargetEncoder

from scipy.stats import zscore

import optuna

# from autofeat import AutoFeatRegressor

# from pystacknet.pystacknet import StackNetRegressor

# from GML.Ghalat_Machine_Learning import Ghalat_Machine_Learning

In [ ]:
def accuracy(y_true,y_pred):
  return 100 * max(0,1-np.sqrt(mean_squared_error(y_true,y_pred)))

In [ ]:
path = '/kaggle/input/he-mlcomp/'

In [ ]:
train = pd.read_csv(path+'Train.csv')
test = pd.read_csv(path+'Test.csv')

In [ ]:
test = train[5000:].copy()
test_labels = test['Attrition_rate']
test.drop(['Attrition_rate'],axis=1,inplace=True)
train = train[:5000].copy()

In [ ]:
all_data = pd.concat([train,test],axis=0)

In [ ]:
all_data['Salary_Now'] = all_data['Pay_Scale'] * 10000
all_data['Salary_In_Next_Year'] = all_data['Salary_Now'] + all_data['Salary_Now'] * (all_data['growth_rate']/100)
all_data['Salary_In_Next_10_Years'] = all_data['Salary_Now'] + all_data['Salary_Now'] * (all_data['growth_rate']/100)*10

In [ ]:
def pos_tos_tsp(s):
    return str(s['Post_Level']) + '_' + str(s['Time_of_service']) + '_' + str(s['Time_since_promotion'])

# all_data['pos_tos_tsp'] = all_data.apply(pos_tos_tsp,axis=1)

In [ ]:
def avg_g_tos(s):
    gr = s['growth_rate']
    tos = s['Time_of_service']
    res = train[(train['growth_rate'] <= (gr + 2)) & (train['growth_rate'] >= (gr - 2)) & 
          (train['Time_of_service'] <= (tos + 2)) & (train['Time_of_service'] >= (tos - 2))].loc[:,'Attrition_rate'].describe()
    if res['mean'] != np.nan:
        return res['mean']
    else:
        return train['Attrition_rate'].mean()

In [ ]:
all_data['avg_g_tos'] = all_data.apply(avg_g_tos,axis=1)

In [ ]:
def UxH(x):
    return str(x['Unit']) + '_' + str(x['Hometown'])

def XxBxH(x):
    return str(x['Compensation_and_Benefits']) + '_' + str(x['Hometown'])

def XxBxU(x):
    return str(x['Compensation_and_Benefits']) + '_' + str(x['Unit'])

In [ ]:
def emp_Type(s):
    if s['Time_of_service'] == 0.0:
        return "New"
    elif s['Time_of_service'] <= 5.0:
        return "Moderate"
    elif s['Time_of_service'] <= 10.0:
        return "Senior"
    else:
        return "SuperSenior"

In [ ]:
def person_Type(s):
    if s['Age'] <= 20:
        return 'Teenager'
    elif s['Age'] <= 40:
        return "Man"
    elif s['Age'] <= 60:
        return "Uncle"
    else:
        return "AboutToDie"

In [ ]:
all_data['Emp_Type'] = all_data.apply(emp_Type,axis=1)

all_data['person_Type'] = all_data.apply(person_Type,axis=1)

all_data['UxH'] = all_data.apply(UxH,axis=1)

In [ ]:
all_data.columns

In [ ]:
all_data['CMP_BF_HT'] = all_data.apply(XxBxH,axis=1)
all_data['CMP_BF_U'] = all_data.apply(XxBxU,axis=1)

In [ ]:
all_data['Perc_Spent_WO_Promotion'] = all_data['Time_since_promotion'] / all_data['Time_of_service']

In [ ]:
# train['zscore'] = zscore(train['Attrition_rate'])

In [ ]:
# train = train[train['zscore'].abs() <= 4].drop(['zscore'],axis=1)

In [ ]:
types = {
    'type0':'M2',
    'type1':'M2',
    'type2':'L2',
    'type4':'L2',
    'type3':'L2'
}

In [ ]:
all_data['COMPType'] = all_data['Compensation_and_Benefits'].map(types)

In [ ]:
test_eid = test['Employee_ID'].copy()

In [ ]:
# pvar = 'Hometown'

In [ ]:
# le = LabelEncoder()
# all_data[pvar] = le.fit_transform(all_data[pvar])

In [ ]:
all_data['Employee_ID'] = all_data['Employee_ID'].apply(lambda x: int(x.split('_')[1]))

In [ ]:
cols = all_data.columns[all_data.dtypes == 'object']
for col in cols:
    all_data[col].interpolate(method='linear',inplace=True)

In [ ]:
# all_data.sort_values('Age',ascending=False,inplace=True)

In [ ]:
def age_bin(s):
    return s['Age']/10

def emp_bin(s):
    if s['Employee_ID'] <= 7200:
        return 'OLD'
    if s['Employee_ID'] <= 18000:
        return 'MID'
    else:
        return 'NEW'

def growth_bin(s):
    if s['growth_rate'] <= 33:
        return 'NOT Satisfy'
    if s['growth_rate'] <= 47:
        return 'Satisfy'
    else:
        return 'High Satisfy'


In [ ]:
all_data['age_bins'] = all_data.apply(age_bin,axis=1)

all_data['emp_bins'] = all_data.apply(emp_bin,axis=1)

all_data['growth_bins'] = all_data.apply(growth_bin,axis=1)

In [ ]:
cat_features = all_data.columns[all_data.dtypes == 'object']

In [ ]:
for cat in cat_features:
    g_var = cat
    minn = dict(all_data.groupby(g_var)['Attrition_rate'].min())
    mediann = dict(all_data.groupby(g_var)['Attrition_rate'].mean())
    maxx = dict(all_data.groupby(g_var)['Attrition_rate'].max())
    stdd = dict(all_data.groupby(g_var)['Attrition_rate'].std())

    all_data[str(cat)+'_Min'] = all_data[g_var].map(minn)
    all_data[str(cat)+'_Avg'] = all_data[g_var].map(mediann)
    all_data[str(cat)+'_Max'] = all_data[g_var].map(maxx)
    all_data[str(cat)+'_Std'] = all_data[g_var].map(stdd)

In [ ]:
cols = all_data.columns
cols.delete(23)
for col in test.columns:
    try:
        all_data[col].replace([np.inf,-np.inf],np.nan,inplace=True)
        all_data[col].interpolate(method='linear',inplace=True)
    except:
        continue

In [ ]:
train = all_data[~pd.isna(all_data['Attrition_rate'])].copy()
test = all_data[pd.isna(all_data['Attrition_rate'])].copy()

In [ ]:
cols = train.columns[train.dtypes == 'object']
encoded = pd.DataFrame([])

X = train[cols]
y = train['Attrition_rate'].copy().values

for tr_in, val_in in KFold(shuffle = True).split(X, y):
    enc = TargetEncoder()
    enc.fit(X.iloc[tr_in], y[tr_in])
    encoded = encoded.append(enc.transform(X.iloc[val_in]))

enc = TargetEncoder()
train[cols] = enc.fit(X[cols],y)
test[cols] = enc.transform(test[cols])
train[cols] = encoded.sort_index()

"""
for col in cols:
  dummies = pd.get_dummies(train[col])
  train.drop([col],axis=1,inplace=True)
  train = pd.concat([train,dummies],axis=1)
  dummies = pd.get_dummies(test[col])
  test.drop([col],axis=1,inplace=True)
  test = pd.concat([test,dummies],axis=1)
"""
"""
for col in test.columns:
    if train[col].isnull().any():
        model = LGBMRegressor(**grid_lgbm)
        t1 = train[~pd.isna(train[col])].copy()
        t1.drop(['Attrition_rate'],axis=1,inplace=True)
        t2 = train[pd.isna(train[col])].copy()
        t2.drop(['Attrition_rate','Age'],axis=1,inplace=True)
        t3 = test[pd.isna(test[col])].copy()
        t3.drop(['Age'],axis=1,inplace=True)
        model.fit(t1.drop([col],axis=1),t1[col])
        train.loc[t2.index,col] = model.predict(t2)
        test.loc[t3.index,col] = model.predict(t3)
"""
"""
for cols in test.columns:
    train[cols].fillna(train[cols].mean(),inplace=True)
    test[cols].fillna(test[cols].mean(),inplace=True)
"""
pass

In [ ]:
"""
ops = ['_mul_','_add_','_sub_','_div_','_log_']
funcs = [np.multiply, np.add, np.subtract, np.divide, np.log1p]
i = 0
cols = test.columns
for func in funcs:
    try:
        for f_1 in cols:
            if i != 4:
                for f_2 in cols:
                    name = str(f_1) + ops[i] + str(f_2)
                    train[name] = func(train[f_1],train[f_2])
                    test[name] = func(test[f_1],test[f_2])
            if i == 4:
                train[f_1 + str('_log')] = func(train[f_1])
                test[f_1 + str('_log')] = func(test[f_1])
    except:
        pass
    i += 1
"""
    
train['UxH^2'] = train['UxH']**2
test['UxH^2'] = test['UxH']**2

train['UxH^3'] = train['UxH']**3
test['UxH^3'] = test['UxH']**3

train['CMP_BF_HT^2'] = train['CMP_BF_HT']**2
test['CMP_BF_HT^2'] = test['CMP_BF_HT']**2

train['CMP_BF_HT^3'] = train['CMP_BF_HT']**3
test['CMP_BF_HT^3'] = test['CMP_BF_HT']**3

train['CMP_BF_U^2'] = train['CMP_BF_U']**2
test['CMP_BF_U^2'] = test['CMP_BF_U']**2

train['CMP_BF_U^3'] = train['CMP_BF_U']**3
test['CMP_BF_U^3'] = test['CMP_BF_U']**3


train['Hometown^2'] = train['Hometown']**2
test['Hometown^2'] = test['Hometown']**2

train['Hometown^3'] = train['Hometown']**3
test['Hometown^3'] = test['CMP_BF_U']**3

In [ ]:
funcs = [np.multiply, np.add, np.subtract, np.divide]

for i, func in enumerate(funcs):
    train['Age_TOS'+str(i+1)] = func(train['Age'],train['Time_of_service'])
    test['Age_TOS'+str(i+1)] = func(test['Age'],test['Time_of_service'])

    train['AT_GR'+str(i+1)] = func(train['Age_TOS'+str(i+1)], train['growth_rate'])
    test['AT_GR'+str(i+1)] = func(test['Age_TOS'+str(i+1)], test['growth_rate'])

    train['TS_GR'+str(i+1)] = func(train['Time_of_service'], train['growth_rate'])
    test['TS_GR'+str(i+1)] = func(test['Time_of_service'], test['growth_rate'])

    train['PS_WLB'+str(i+1)] = func(train['Pay_Scale'], train['Work_Life_balance'])
    test['PS_WLB'+str(i+1)] = func(test['Pay_Scale'], test['Work_Life_balance'])

    train['Age_TSP'+str(i+1)] = func(train['Age'], train['Time_since_promotion'])
    test['Age_TSP'+str(i+1)] = func(test['Age'],test['Time_since_promotion'])

    train['Age_EMP'+str(i+1)] = func(train['Age'], train['Employee_ID'])
    test['Age_EMP'+str(i+1)] = func(test['Age'], test['Employee_ID'])

    train['TOS_EMP'+str(i+1)] = func(train['Time_of_service'],train['Employee_ID'])
    test['TOS_EMP'+str(i+1)] = func(test['Time_of_service'],test['Employee_ID'])

    train['ATOS_EMP'+str(i+1)] = func(train['Age_TOS'+str(i+1)], train['Employee_ID'])
    test['ATOS_EMP'+str(i+1)] = func(test['Age_TOS'+str(i+1)], test['Employee_ID'])
    
var_names = ['VAR'+str(i) for i in range(1,8)]
sums_tr = [] 
subs_tr = [] 
mul_tr = [] 
div_tr = []
sums_ts = []
subs_ts = []
mul_ts = []
div_ts = []
for row in train[var_names].itertuples():
    sums_tr.append(row[1]+row[2]+row[3]+row[4]+row[5]+row[6]+row[7])
    subs_tr.append(row[1]-row[2]-row[3]-row[4]-row[5]-row[6]-row[7])
    mul_tr.append(row[1]*row[2]*row[3]*row[4]*row[5]*row[6]*row[7])
    div_tr.append(row[1]/row[2]/row[3]/row[4]/row[5]/row[6]/row[7])
    
for row2 in test[var_names].itertuples():
    sums_ts.append(row[1]+row[2]+row[3]+row[4]+row[5]+row[6]+row[7])
    subs_ts.append(row[1]-row[2]-row[3]-row[4]-row[5]-row[6]-row[7])
    mul_ts.append(row[1]*row[2]*row[3]*row[4]*row[5]*row[6]*row[7])
    div_ts.append(row[1]/row[2]/row[3]/row[4]/row[5]/row[6]/row[7])
    
train['sum_VARS'] = sums_tr
train['sub_VARS'] = subs_tr
train['mul_VARS'] = mul_tr
train['div_VARS'] = div_tr
test['sum_VARS'] = sums_ts
test['sub_VARS'] = subs_ts
test['mul_VARS'] = mul_ts
test['div_VARS'] = div_ts

train['mulvars_gr'] = train['growth_rate'] * train['mul_VARS']
test['mulvars_gr'] = test['growth_rate'] * test['mul_VARS']


for col in test.columns:
    try:
        train[col].replace([np.inf,-np.inf],np.nan,inplace=True)
        test[col].replace([np.inf,-np.inf],np.nan,inplace=True)
        train[col].interpolate(method='linear',inplace=True)
        test[col].interpolate(method='linear',inplace=True)
    except:
        continue

In [ ]:
grid_bayesian = {
    'n_iter':10000,
    'tol':6.42e-06,
    'alpha_1':0.0001,
    'alpha_2':6.58e-05,
    'lambda_1':0.999,
    'lambda_2':2.24e-05,
    'compute_score':True,
    'fit_intercept':True,
    'normalize':False
}

grid_ridge = {
    'max_iter':90000,
    'alpha': 0.23143352151759083, 
    'fit_intercept': True, 
    'normalize': False, 
    'tol': 0.0005243805101392493, 
    'solver': 'saga'
}

In [ ]:
rbs = RobustScaler()

In [ ]:
X = train.drop(['Attrition_rate'],axis=1)
y = train['Attrition_rate'].copy().values

In [ ]:
test.drop(['Attrition_rate'],axis=1,inplace=True)

In [ ]:
# gml = Ghalat_Machine_Learning()

# gml.GMLRegressor(X, y, test_Size = 0.4, metric = accuracy)

In [ ]:
nb = BayesianRidge()
nb.fit(X, y)
"""
fi = pd.DataFrame()
fi['A'] = X.columns.values
fi['B'] = lgbm.feature_importances_
"""
pass

In [ ]:
r = permutation_importance(nb, X, y,
                            n_repeats=10,
                            random_state=0)

In [ ]:
cols_to_d = []

In [ ]:
thresh = 0.01

In [ ]:
for i in r.importances_mean.argsort()[::-1]:
    try:
        if train.columns[i] == 'Attrition_rate':
            continue
        print(f"{train.columns[i]:<8}\t"
           f"{r.importances_mean[i]:.3f}"
           f" +/- {r.importances_std[i]:.3f}")
        if r.importances_mean[i] <= thresh:
            cols_to_d.append(train.columns[i])
    except:
        continue

In [ ]:
X.drop(cols_to_d,axis=1,inplace=True)
test.drop(cols_to_d,axis=1,inplace=True)

In [ ]:
"""
from statsmodels.stats.outliers_influence import variance_inflation_factor    

def calculate_vif_(X, thresh=5.0):
    variables = list(range(X.shape[1]))
    dropped = True
    while dropped:
        dropped = False
        vif = [variance_inflation_factor(X.iloc[:, variables].values, ix)
               for ix in range(X.iloc[:, variables].shape[1])]

        maxloc = vif.index(max(vif))
        if max(vif) > thresh:
            print('dropping \'' + X.iloc[:, variables].columns[maxloc] +
                  '\' at index: ' + str(maxloc))
            del variables[maxloc]
            dropped = True

    print('Remaining variables:')
    print(X.columns[variables])
    return X.iloc[:, variables]
"""
pass

In [ ]:
# X = calculate_vif_(X, thresh=10)
# test = test[X.columns]

In [ ]:
X = pd.DataFrame(rbs.fit_transform(X, y))
test = pd.DataFrame(rbs.transform(test))

In [ ]:
def opti(trial):
    p1 = trial.suggest_loguniform('p1',1,100)
    p2 = trial.suggest_loguniform('p2',1,100)
    p3 = trial.suggest_loguniform('p3',1,100)
    p4 = trial.suggest_loguniform('p4',1,100)
    p5 = trial.suggest_loguniform('p5',1,100)
    p6 = trial.suggest_loguniform('p6',1,100)
    
    s1 = trial.suggest_categorical('s1',[True,False])
    s2 = trial.suggest_categorical('s2',[True,False])
    s3 = trial.suggest_categorical('s3',[True,False])
    s4 = trial.suggest_categorical('s4',[True,False])
    s5 = trial.suggest_categorical('s5',[True,False])
    
    if s1:
        p1 = -p1
    if s2:
        p2 = -p2
    if s3:
        p3 = -p3
    if s4:
        p4 = -p4
    if s5:
        p5 = -p5
    
    scores = []

    for tr_in, val_in in KFold(n_splits = 5, shuffle=True).split(X, y):
        br = BayesianRidge()
        ridge = Ridge(**grid_ridge)
        lasso = LassoLars()
        lr = LinearRegression()
        en = ElasticNet()
        
        br.fit(X.iloc[tr_in], np.log1p(y[tr_in]))
        ridge.fit(X.iloc[tr_in], np.log1p(y[tr_in]))
        lasso.fit(X.iloc[tr_in], np.log1p(y[tr_in]))
        lr.fit(X.iloc[tr_in], np.log1p(y[tr_in]))
        en.fit(X.iloc[tr_in], np.log1p(y[tr_in]))
        
        preds = (br.predict(X.iloc[val_in]) * p1 + ridge.predict(X.iloc[val_in]) * p2 + 
                lasso.predict(X.iloc[val_in]) * p3 + lr.predict(X.iloc[val_in]) * p4 + 
                en.predict(X.iloc[val_in]) * p5 )
        preds = np.expm1(preds)
        scores.append(accuracy(y[val_in],preds))
    return -np.mean(scores)

In [ ]:
study = optuna.create_study()

In [ ]:
study.optimize(opti)

In [ ]:
ss = pd.DataFrame()
ss['Employee_ID'] = test_eid
ss['Attrition_rate'] = np.nan

In [ ]:
br = BayesianRidge()
ridge = Ridge(**grid_ridge)
lasso = LassoLars()
lr = LinearRegression()
en = ElasticNet()
knn = KNeighborsRegressor()

In [ ]:
scores = []

for tr_in, val_in in KFold(shuffle=True).split(X, y):
    stack = StackingCVRegressor(regressors=(br, lasso, ridge, en, knn),
                             meta_regressor=lr,
                             random_state=42)
    stack.fit(X.iloc[tr_in], np.log1p(y[tr_in]))
    preds = stack.predict(X.iloc[val_in]) 
    preds = np.expm1(preds)
    scores.append(accuracy(y[val_in],preds))

In [ ]:
scores

In [ ]:
np.mean(scores)

In [ ]:
stack = StackingCVRegressor(regressors=(br, lasso, ridge, en, knn),
                             meta_regressor=lr,
                             random_state=42)

stack.fit(X, np.log1p(y))

In [ ]:
ss['Attrition_rate'] = np.expm1(stack.predict(test))

In [ ]:
# accuracy(ss['Attrition_rate'],test_labels)

In [ ]:
ss.to_csv('submission.csv',index=False)

In [ ]:
high_sub = pd.read_csv('/kaggle/input/81358-he/81358.csv')

In [ ]:
sub = pd.DataFrame()

In [ ]:
sub['Employee_ID'] = ss['Employee_ID']

In [ ]:
high_sub = pd.read_csv('/kaggle/input/81364-he/81364.csv')

In [ ]:
err = np.abs(high_sub['Attrition_rate'] - ss['Attrition_rate'] )

In [ ]:
new_subs = []
for r1 , r2 , er in zip( high_sub['Attrition_rate'].values , ss['Attrition_rate'].values , err ):
    if er > 0.02:
        new_subs.append( 0.95*r1 + 0.05*r2 )
    else:
        new_subs.append( r1 - er/8 )

In [ ]:
sub['Attrition_rate'] = (high_sub['Attrition_rate'] - err/18 )

In [ ]:
# sub['Attrition_rate'] = new_subs

In [ ]:
sub

In [ ]:
sub.to_csv('submission.csv',index=False)